In [1]:

from pathlib import Path

from datasets import Valentini

dataset = Valentini()

/home/maksim/.local/lib/python3.10/site-packages/torchaudio/compliance/kaldi.py:22: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  EPSILON = torch.tensor(torch.finfo(torch.float).eps)


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
noisy , clean = dataset[0]

In [2]:
!which python

/home/maksim/miniconda3/bin/python


In [ ]:
from IPython.display import Audio
Audio(noisy,)

In [12]:
from datasets import Valentini

dataset = Valentini()


ModuleNotFoundError: No module named 'matplotlib'

In [2]:
dataset_path = Path('/media/public/datasets/denoising/DS_10283_2791')
clean_path = dataset_path / 'clean_testset_wav'
noisy_path = dataset_path / 'noisy_testset_wav'

In [3]:
clean_wavs = list(clean_path.glob("*"))
noisy_wavs = list(noisy_path.glob("*"))

In [4]:
len(clean_wavs), len(noisy_wavs)

(824, 824)

In [5]:
from IPython.display import Audio, display

In [6]:
def display_pair(i):
    display(Audio(noisy_wavs[i], rate=48000))
    display(Audio(clean_wavs[i], rate=48000))
display_pair(-1)

- SDR https://torchmetrics.readthedocs.io/en/stable/audio/signal_distortion_ratio.html
- SI-SNR https://torchmetrics.readthedocs.io/en/stable/audio/scale_invariant_signal_noise_ratio.html?highlight=Si-SNR

/home/maksim/miniconda3/bin/python


In [8]:
from torchmetrics.audio.pesq import PerceptualEvaluationSpeechQuality
from torchmetrics.audio.stoi import ShortTimeObjectiveIntelligibility
import torch
import torchaudio
import torchmetrics
from denoisers.SpectralGating import SpectralGating


class Metrics:
    def __init__(self, rate=16000):
        self.nb_pesq = PerceptualEvaluationSpeechQuality(rate, 'wb')
        self.stoi = ShortTimeObjectiveIntelligibility(rate, False)
    def calculate(self, preds, target):
        return {'PESQ': self.nb_pesq(preds, target), 
                'STOI': self.stoi(preds, target)}

def load_wav(path):
    wav, org_sr = torchaudio.load(path)
    wav = torchaudio.functional.resample(wav, orig_freq=org_sr, new_freq=16000)
    return wav




model = SpectralGating()
metrics = Metrics()

clean_wav = load_wav(clean_wavs[0])
noisy_wav = load_wav(noisy_wavs[0])
denoised = model(noisy_wav)


ModuleNotFoundError: No module named 'torchmetrics'

In [ ]:
metrics.calculate(noisy_wav, clean_wav)

In [ ]:
metrics.calculate(denoised, clean_wav)

In [ ]:
from tqdm import tqdm
mean_scores_ideal = {'PESQ': 0,'STOI': 0}
mean_scores_model =  {'PESQ': 0, 'STOI': 0}

for clean_path, noisy_path in tqdm(zip(clean_wavs[:10], noisy_wavs[:10])):
    clean_wav = load_wav(clean_path)
    noisy_wav = load_wav(noisy_path)
    denoised_wav = model(noisy_wav)
    
    scores_ideal = metrics.calculate(noisy_wav, clean_wav)
    scores_model = metrics.calculate(noisy_wav, denoised_wav)
    
    mean_scores_ideal['PESQ'] += scores_ideal['PESQ']
    mean_scores_ideal['STOI'] += scores_ideal['STOI']
    
    mean_scores_model['PESQ'] += scores_model['PESQ']
    mean_scores_model['STOI'] += scores_model['STOI']

mean_scores_ideal['PESQ'] = mean_scores_ideal['PESQ'] / len(clean_wavs)
mean_scores_ideal['STOI'] = mean_scores_ideal['STOI'] / len(clean_wavs)
mean_scores_model['PESQ'] = mean_scores_model['PESQ'] / len(clean_wavs)
mean_scores_model['STOI'] = mean_scores_model['STOI'] / len(clean_wavs)

In [ ]:
mean_scores_ideal

In [ ]:
mean_scores_model

In [3]:
def inference(i):
    prediction = model(noisy_wavs[i])
    display(Audio(noisy_wavs[i],rate=48000))
    display(Audio(clean_wavs[i],rate=48000))
    display(Audio(prediction,rate=48000))
    
inference(5)

NameError: name 'model' is not defined

In [ ]:
from huggingsound import SpeechRecognitionModel

model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-spanish")
audio_paths = ["/path/to/file.mp3", "/path/to/another_file.wav"]

transcriptions = model.transcribe(audio_paths)
